In [1]:
import pandas as pd
import numpy as np
import random
import string
import datetime
import csv

In [2]:
files = [
    "export/abschluss2019.csv",
    "export/abschluss2020.csv",
    "export/abschluss2021-0.csv",
    "export/abschluss2021-1.csv",
    "export/abschluss2022.csv",
    "export/abschluss2007.csv",
    "export/abschluss2017.csv",
    "export/abschluss2018.csv",
    "export/externe.csv",
]

Daten einlesen

In [3]:
dfs = []

for file in files:
    print(f"Reading {file}")
    dfs.append(pd.read_csv(file, sep=",", encoding="ISO-8859-1"))

df = pd.concat(dfs, ignore_index=True)

Reading export/abschluss2019.csv
Reading export/abschluss2020.csv
Reading export/abschluss2021-0.csv
Reading export/abschluss2021-1.csv
Reading export/abschluss2022.csv
Reading export/abschluss2007.csv
Reading export/abschluss2017.csv
Reading export/abschluss2018.csv
Reading export/externe.csv


Struktur ausgeben

In [4]:
print(f"Users: {len(df)}")
print(f"Users: {df.columns}")

Users: 2985
Users: Index(['Name', 'Type', 'Description', 'Display Name', 'E-Mail Address',
       'Company', 'Job Title', 'Last Name', 'First Name', 'User Logon Name',
       'Department'],
      dtype='object')


Nicht benötigte Zeilen entfernen(werden später neu generiert)

In [5]:
columns_to_remove = ['Name', 'Display Name', 'E-Mail Address', 'Company', 'Job Title', 'User Logon Name', 'Department']
print(f"Users: Removing columns: {columns_to_remove}")
df = df.drop(columns=columns_to_remove)
print(f"Users: {df.columns}")

Users: Removing columns: ['Name', 'Display Name', 'E-Mail Address', 'Company', 'Job Title', 'User Logon Name', 'Department']
Users: Index(['Type', 'Description', 'Last Name', 'First Name'], dtype='object')


#### Display Name generieren
- Vorname + Nachname

In [6]:
# create a column "Display Name" Value = "First Name" + " " + "Last Name"
df["Display Name"] = df["First Name"] + " " + df["Last Name"]
print(f"Users: {df.head()}")

Users:                Type         Description  Last Name First Name  \
0  Heidemann Adrian    Oberstufe extern  Heidemann     Adrian   
1      Alena Herwig   Oberstufe - Matur     Herwig      Alena   
2      Alena Kanone   Oberstufe - Matur     Kanone      Alena   
3        Alena Kolb         deaktiviert       Kolb      Alena   
4       Heinz Alexa    Oberstufe extern      Heinz      Alexa   

       Display Name  
0  Adrian Heidemann  
1      Alena Herwig  
2      Alena Kanone  
3        Alena Kolb  
4       Alexa Heinz  


#### E-Mail generieren
- Vorname + Nachname + @gertzstein.local
- Sonderzeichen entfernen

In [7]:
df["mail"] = df["First Name"].str.lower() + "." + df["Last Name"].str.lower() + "@gertzstein.local"
df["mail"] = df["mail"].str.replace(" ", "-")
df["mail"] = df["mail"].str.replace("ä", "ae")
df["mail"] = df["mail"].str.replace("ü", "ue")
df["mail"] = df["mail"].str.replace("ö", "oe")

print(f"Users: {df.head()}")

Users:                Type         Description  Last Name First Name  \
0  Heidemann Adrian    Oberstufe extern  Heidemann     Adrian   
1      Alena Herwig   Oberstufe - Matur     Herwig      Alena   
2      Alena Kanone   Oberstufe - Matur     Kanone      Alena   
3        Alena Kolb         deaktiviert       Kolb      Alena   
4       Heinz Alexa    Oberstufe extern      Heinz      Alexa   

       Display Name                               mail  
0  Adrian Heidemann  adrian.heidemann@gertzstein.local  
1      Alena Herwig      alena.herwig@gertzstein.local  
2      Alena Kanone      alena.kanone@gertzstein.local  
3        Alena Kolb        alena.kolb@gertzstein.local  
4       Alexa Heinz       alexa.heinz@gertzstein.local  


#### Username generieren

"B" + 6 Zufallszahlen
einmalig

In [8]:
df["username"] = ""

def generate_username():
    username = "B" + str(np.random.randint(100000, 999999))
    while username in df["username"].values:
        username = "B" + str(np.random.randint(100000, 999999))
    return username
    
df["username"] = df["username"].apply(lambda x: generate_username())
print(f"Users: {df.head()}")

Users:                Type         Description  Last Name First Name  \
0  Heidemann Adrian    Oberstufe extern  Heidemann     Adrian   
1      Alena Herwig   Oberstufe - Matur     Herwig      Alena   
2      Alena Kanone   Oberstufe - Matur     Kanone      Alena   
3        Alena Kolb         deaktiviert       Kolb      Alena   
4       Heinz Alexa    Oberstufe extern      Heinz      Alexa   

       Display Name                               mail username  
0  Adrian Heidemann  adrian.heidemann@gertzstein.local  B296262  
1      Alena Herwig      alena.herwig@gertzstein.local  B801639  
2      Alena Kanone      alena.kanone@gertzstein.local  B489413  
3        Alena Kolb        alena.kolb@gertzstein.local  B848886  
4       Alexa Heinz       alexa.heinz@gertzstein.local  B192210  


Duplikate entfernen

In [9]:
print(f"Number of Users: {len(df)}")
df = df.drop_duplicates(subset=["mail"])
print(f"Number of Users: {len(df)}")

Number of Users: 2985
Number of Users: 2982


Speichern

In [10]:
now = datetime.datetime.now()
date_time = now.strftime("%Y-%m-%d_%H-%M-%S")
filename_csv = date_time + "_students" + ".csv"

df.to_csv(filename_csv, index=False)
print(f"File {filename_csv} saved")


File 2024-03-11_13-34-59_students.csv saved


In [11]:
with open(filename_csv, 'r') as file :
  filedata = file.read()
  
filedata = filedata.replace(',', '";"')

filedata = '\n'.join(['"' + line + '"' for line in filedata.splitlines()])


with open(filename_csv, 'w') as file:
    file.write(filedata)
    
print(f"File {filename_csv} saved")

File 2024-03-11_13-34-59_students.csv saved
